# Predicting House Sale Price


## Introduction

We'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can read more about why the data was collected [here](https://doi.org/10.1080/10691898.2011.11889627). You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

Let's start by setting up a pipeline of functions that will let us quickly iterate on different models.

<img src="pipeline.svg" width="200">


In [26]:
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)  

df = pd.read_csv('AmesHousing.tsv',delimiter="\t")
df.head(5)

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [27]:
df.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
      

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

In [29]:
df.describe(include='all')

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
count,2930.00000,2.930000e+03,2930.000000,2930,2440.000000,2930.000000,2930,198,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930,2930.000000,2930.000000,2930.000000,2930.000000,2930,2930,2930,2930,2907,2907.000000,2930,2930,2930,2850,2850,2847,2850,2929.000000,2849,2929.000000,2929.000000,2929.000000,2930,2930,2930,2929,2930.000000,2930.000000,2930.000000,2930.000000,2928.000000,2928.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930,2930.000000,2930,2930.000000,1508,2773,2771.000000,2771,2929.000000,2929.000000,2771,2771,2930,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,13,572,106,2930.000000,2930.000000,2930.000000,2930,2930,2930.000000
unique,NaN,NaN,NaN,7,NaN,NaN,2,2,4,4,3,5,3,28,9,8,5,8,NaN,NaN,NaN,NaN,6,8,16,17,5,NaN,4,5,6,5,5,4,6,NaN,6,NaN,NaN,NaN,6,5,2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,8,NaN,5,6,NaN,3,NaN,NaN,5,5,3,NaN,NaN,NaN,NaN,NaN,NaN,4,4,5,NaN,NaN,NaN,10,6,NaN
top,NaN,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN,NaN,Gable,CompShg,VinylSd,VinylSd,None,NaN,TA,TA,PConc,TA,TA,No,GLQ,NaN,Unf,NaN,NaN,NaN,GasA,Ex,Y,SBrkr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA,NaN,Typ,NaN,Gd,Attchd,NaN,Unf,NaN,NaN,TA,TA,Y,NaN,NaN,NaN,NaN,NaN,NaN,Ex,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,NaN,2273,NaN,NaN,2918,120,1859,2633,2927,2140,2789,443,2522,2900,2425,1481,NaN,NaN,NaN,NaN,2321,2887,1026,1015,1752,NaN,1799,2549,1310,1283,2616,1906,859,NaN,2499,NaN,NaN,NaN,2885,1495,2734,2682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1494,NaN,2728,NaN,744,1731,NaN,1231,NaN,NaN,2615,2665,2652,NaN,NaN,NaN,NaN,NaN,NaN,4,330,95,NaN,NaN,NaN,2536,2413,NaN
mean,1465.50000,7.144645e+08,57.387372,NaN,69.224590,10147.921843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.094881,5.563140,1971.356314,1984.266553,NaN,NaN,NaN,NaN,NaN,101.896801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442.629566,NaN,49.722431,559.262547,1051.614544,NaN,NaN,NaN,NaN,1159.557679,335.455973,4.676792,1499.690444,0.431352,0.061134,1.566553,0.379522,2.854266,1.044369,NaN,6.443003,NaN,0.599317,NaN,NaN,1978.132443,NaN,1.766815,472.819734,NaN,NaN,NaN,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,NaN,NaN,NaN,50.635154,6.216041,2007.790444,NaN,NaN,180796.060068
std,845.96247,1.887308e+08,42.638025,NaN,23.365335,7880.017759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.411026,1.111537,30.245361,20.860286,NaN,NaN,NaN,NaN,NaN,179.112611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455.590839,NaN,169.168476,439.494153,440.615067,NaN,NaN,NaN,NaN,391.890885,428.395715,46.310510,505.508887,0.524820,0.245254,0.552941,0.502629,0.827731,0.214076,NaN,1.572964,NaN,0.647921,NaN,NaN,25.528411,NaN,0.760566,215.046549,NaN,NaN,NaN,126.361562,67.483400,64.139059,25.141331,56.087370,35.597181,NaN,NaN,NaN,566.344288,2.714492,1.316613,NaN,NaN,79886.692357
min,1.00000,5.263011e+08,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1872.000000,1950.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.

In [30]:
def  transform_features(df):
    return df
    
def select_features(df):
    return   df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    
    data = select_features(df)
    data = data.select_dtypes(include=['float64','int64'])
    train = data[0:1460]
    test  = data[1460:] 

    features = data.columns.drop('SalePrice')

    lr = linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = mse ** (1/2)
    return rmse

transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse    

57088.25161263909

## Feature Engineering

1. remove features that we don't want to use in the model, just based on the number of missing values or data leakage

transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc)

create new features by combining other features


Handle missing values:
* All columns:
    * Drop any with 5% or more missing values for now.
* Text columns:
    * Drop any with 1 or more missing values for now.
* Numerical columns:
    * For columns with missing values, fill in with the mode common value in that column

Drop columns that:
* that aren't useful for ML
* leak data about the final sale, read more about columns here    

In [31]:
# 1: All columns: Drop any with 5% or more missing values for now.

null_counts = df.isnull().sum()
length = len(df) * 0.05
print(df.shape)
df = df[ null_counts[null_counts < length].index]
print(df.shape)

(2930, 82)
(2930, 71)


In [32]:
# 2.Text columns: Drop any with 1 or more missing values for now.
text_null_counts =df.select_dtypes(include=['object']).isnull().sum()
text_null_cols = text_null_counts[text_null_counts > 0 ].index
df = df.drop(columns=text_null_cols)
print(df.shape)

(2930, 64)


In [33]:
# 3.Numerical columns: For columns with missing values, fill in with the most common value in that column
    
num_null_counts =df.select_dtypes(include=['float64',"int64"]).isnull().sum()
num_null_cols = num_null_counts[num_null_counts > 0].index.tolist()
print(num_null_cols)

for col in num_null_cols:
    col_mode = df[col].mode()
    df[col] = df[col].fillna(col_mode[0])

['Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Garage Cars', 'Garage Area']


In [34]:
# 4. Create new columns
df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']

## Drop rows with negative values for both of these new features
df = df [ (df['Years Before Sale']>0)  & (df['Years Since Remod']>0)]

## No longer need original year columns
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

In [35]:
## Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

Let's update transform_features() and read in data again

In [36]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

def  transform_features(df):
    # 1: All columns: Drop any with 5% or more missing values for now.
    null_counts = df.isnull().sum()
    length = len(df) * 0.05
    df = df[ null_counts[null_counts < length].index]

    # 2.Text columns: Drop any with 1 or more missing values for now.
    text_null_counts =df.select_dtypes(include=['object']).isnull().sum()
    text_null_cols = text_null_counts[text_null_counts > 0 ].index
    df = df.drop(columns=text_null_cols)    
    
    # 3.Numerical columns: For columns with missing values, fill in with the most common value in that column
    num_null_counts =df.select_dtypes(include=['float64',"int64"]).isnull().sum()
    num_null_cols = num_null_counts[num_null_counts > 0].index.tolist()
    for col in num_null_cols:
        col_mode = df[col].mode()
        df[col] = df[col].fillna(col_mode[0])    

    # 4. Create new columns
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df = df [(df['Years Before Sale']>0)  & (df['Years Since Remod']>0)]
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)        

    # 5. Drop columns that aren't useful or leak info about the final sale
    df = df.drop(["PID", "Order","Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

def select_features(df):
    return   df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    
    data = select_features(df)
    data = data.select_dtypes(include=['float64','int64'])
    train = data[:1460]
    test  = data[1460:] 

    features = data.columns.drop('SalePrice')

    lr = linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = mse ** (1/2)
    return rmse

transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse    

47125.95445367698

## Feature Selection


In [37]:
num_df =transform_df.select_dtypes(include=['float64','int64'])
corrmat =transform_df.corr()['SalePrice']
corrmat[corrmat >0.4]

Overall Qual     0.785137
Mas Vnr Area     0.469301
BsmtFin SF 1     0.450672
Total Bsmt SF    0.632745
1st Flr SF       0.623102
Gr Liv Area      0.724094
Full Bath        0.541690
TotRms AbvGrd    0.476159
Fireplaces       0.488419
Garage Cars      0.622464
Garage Area      0.608802
SalePrice        1.000000
Name: SalePrice, dtype: float64

In [38]:
## Drop columns with less than 0.4 correlation with SalePrice
transform_df = transform_df.drop(corrmat[corrmat < 0.4].index, axis=1)
transform_df.columns

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area',
       'Exter Qual', 'Exter Cond', 'Foundation', 'BsmtFin SF 1',
       'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air', '1st Flr SF',
       'Gr Liv Area', 'Full Bath', 'Kitchen Qual', 'TotRms AbvGrd',
       'Functional', 'Fireplaces', 'Garage Cars', 'Garage Area', 'Paved Drive',
       'SalePrice'],
      dtype='object')

* Which columns in the data frame should be converted to the categorical data type? 
  * We will drop categorical column has more than 10 of unique values
  * Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).
* Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

In [39]:
## Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [40]:
###  Find and drop categorical column has more than 10 of unique values
transform_cat_cols =[]
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda x : len(x.value_counts()))

## Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)


In [41]:
## Select just the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])

for col in text_cols: 
    transform_df[col] = transform_df[col].astype('category')

## Create dummy columns and add back to the dataframe!
# transform_df = pd.concat([
#     transform_df, 
#     pd.get_dummies(transform_df.select_dtypes(include=['category']))
# ], axis=1).drop(text_cols,axis=1)    
    
## Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df,
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis =1).drop(text_cols,axis=1)


* Update select_features() using the above code

* Update train_and_test(), implement k-fold cross validation using k folds:



In [42]:
def  transform_features(df):  # feature engineering
    # 1: All columns: Drop any with 5% or more missing values for now.
    null_counts = df.isnull().sum()
    length = len(df) * 0.05
    df = df[ null_counts[null_counts < length].index]

    # 2.Text columns: Drop any with 1 or more missing values for now.
    text_null_counts =df.select_dtypes(include=['object']).isnull().sum()
    text_null_cols = text_null_counts[text_null_counts > 0 ].index
    df = df.drop(columns=text_null_cols)    
    
    # 3.Numerical columns: For columns with missing values, fill in with the most common value in that column
    num_null_counts =df.select_dtypes(include=['float64',"int64"]).isnull().sum()
    num_null_cols = num_null_counts[num_null_counts > 0].index.tolist()
    for col in num_null_cols:
        col_mode = df[col].mode()
        df[col] = df[col].fillna(col_mode[0])    

    # 4. Create new columns
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df = df [(df['Years Before Sale']>0)  & (df['Years Since Remod']>0)]
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)        

    # 5. Drop columns that aren't useful or leak info about the final sale
    df = df.drop(["PID", "Order","Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10): # feature selection
    num_df = df.select_dtypes(include=['float64','int64'])
    corrmat =num_df.corr()['SalePrice']
    
    ## Drop columns with less than 'coeff_threshold' correlation with SalePrice
    df = df.drop(corrmat[corrmat < coeff_threshold].index, axis=1)
    
    ## Create a list of column names from documentation that are *meant* to be categorical
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                        "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                        "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                        "Misc Feature", "Sale Type", "Sale Condition"]
    
    ###  Find and drop categorical column has more than 'uniq_threshold' of unique values
    transform_cat_cols =[]
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
    uniqueness_counts = df[transform_cat_cols].apply(lambda x : len(x.value_counts()))
    
    ## Aribtrary cutoff of 10 unique values (worth experimenting)
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > uniq_threshold].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    ## Select just the remaining text columns and convert to categorical
    text_cols = df.select_dtypes(include=['object'])
    
    for col in text_cols: 
        df[col] = df[col].astype('category')

    ## Create dummy columns and add back to the dataframe!
    df = pd.concat([
        df,
        pd.get_dummies(df.select_dtypes(include=['category']))
    ], axis =1).drop(text_cols,axis=1)
    
    return df

def train_and_test(df, k=0):
    
    data = df.select_dtypes(include=['float64','int64'])
    features = data.columns.drop('SalePrice')    
    lr = linear_model.LinearRegression()   
    
    if k==0:
        train = data[:1460]
        test  = data[1460:] 
        lr.fit(train[features],train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], predictions)
        rmse = mse ** (1/2)
        return rmse
    if  k==1:
        
        datta = data.sample(frac=1)
        train = data[:1460]
        test  = data[1460:] 
        
        lr.fit(train[features],train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], predictions)
        rmse1 = mse ** (1/2)

        lr.fit(test[features],test['SalePrice'])
        predictions = lr.predict(train[features])
        mse = mean_squared_error(train['SalePrice'], predictions)
        rmse2 = mse ** (1/2)
        print (rmse1)
        print (rmse2)
        return (rmse1+ rmse2)/2   
    else:
        
        kf = KFold(k, shuffle=True, random_state=1)
        rmse_values = []
        
        for train_index, test_index in kf.split(df):
            train = data.iloc[train_index]
            test  = data.iloc[test_index] 

            lr.fit(train[features],train['SalePrice'])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test['SalePrice'], predictions)
            rmse_values.append (mse ** (1/2))
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
       
    
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df,5)

rmse 

[28559.253979995625, 27617.41549010686, 34087.46202251305, 26972.30664851595, 30659.84840825993]


29579.25730987828